# Generating Images

The entire approach hinges on the concept of converting audio into images.
This notebook generates all the spectrograms for the different files.

Additionally, it also seperates out the audio data into a training and test sets.
Its important to sperate the audio files and not the generated spectrums.

---

This notebook needs very little input. Once you added a new class to the ``AudioData`` folder you can just run all the cells one after the other. The last cells will take a while depended on how much audio data you have in the ``AudioData`` Folder. 

Have a look at the ``AudioData`` folder and the ``GeneratedData`` folder as its processing.



In [1]:
Type="GENERATE"  #For loading our helper functions we tell the sytem that we are planning to Generate images
from subprocess import Popen # We need a specific library for running the images generation in parrallel
exec(open("../helperFunctions.py","rb").read())

The next cell defines the folder paths to both the source folder where the audio is stored and the folder where the generated images should go.

At a later stage you can define different folders for specifc projects or the like. For now just keep it.

In [2]:
SOURCE_DATA_ROOT='../AudioData/' 
GENERATED_DATA_ROOT='../GeneratedData/'

The next cell does all the work. 

In general what we do is, we split each class i.e. each folder into a rough 80%/20% split. 80% of the audio files get put into a training folder, while 20% get put into a testing folder. You can see that structure in the ```GeneratedData``` folder once its done. But, we don't just copy the audio files we use librosas mel spectrum function to generate a spectrogram and store that in the respective training or testing folder. [Librosa docs](https://librosa.org/doc/latest/generated/librosa.feature.melspectrogram.html?highlight=mel#librosa.feature.melspectrogram)

In [25]:
ToDoList=[]
SourceFoldersLabels = [f.path for f in os.scandir(SOURCE_DATA_ROOT) if f.is_dir()]
for path in SourceFoldersLabels:
    #find the audio files at the path
    FileList = np.array([f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".aif") or f.name.endswith(".flac") or f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))])
    Label = os.path.split(path)[-1]
    #print("label", Label)
    
    #Make the output folder files if they don't exist already
    OutFolderTrain = os.path.join(GENERATED_DATA_ROOT,Label,'train')
    OutFolderTest = os.path.join(GENERATED_DATA_ROOT,Label,'test')
    if not os.path.exists(OutFolderTrain):
        os.makedirs(OutFolderTrain)
    if not os.path.exists(OutFolderTest):
        os.makedirs(OutFolderTest)
        
    #Shake up the file list and separate into train and test. Put them on ToDoList
    np.random.shuffle(FileList)
    trainCount =np.int(np.floor(0.8*FileList.shape[0]))
    train_set = FileList[:trainCount]
    test_set = FileList[trainCount:]
    for f in train_set:
        ToDoList.append((os.path.abspath(f),os.path.abspath(OutFolderTrain)))
    for f in test_set:
        ToDoList.append((os.path.abspath(f),os.path.abspath(OutFolderTest)))
    print("Finished class",Label,". Going to the next.")

#print('ToDoList', ToDoList)

## Not sure what's happening here. Presumably iterating through files and making spectrograms?
## Does a batch of 12 and then the next batch.
Commands = [[sys.executable, "../helperFunctions.py",t[0],t[1]] for t in ToDoList]
print("Done Creating our ToDoList. I'll start computing now, hold on.")

batch=[]
for i in range(len(Commands)):
    batch.append(Commands[i])
    #print ('Commands', i, Commands[i])
    if(len(batch)>=12):
        #print('len(batch)', len(batch)) ## <= To optimize you can type in here how many CPU cores/threads you have 
        procs = [Popen(j) for j in batch]
        #print('procs', procs)
        for p in procs:
            p.wait()
        batch=[]

procs = [ Popen(j) for j in batch ]
for p in procs:
    p.communicate() 
    
print("All Done.")

Finished class s . Going to the next.
Finished class h . Going to the next.
Finished class b . Going to the next.
Done Creating our ToDoList. I'll start computing now, hold on.
All Done.
